In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [8]:
# The fields show the original variable names alongside their aliases and category meanings.

fields = {

    # Basic demographic variables
    'V161342': 'gender', # 1:'male',2:'female'
    'V161310x':'race',  # 1:'white',2:'black',3:'asian',5:'hispanic'
    'V161267': 'age',  # age in years, negative numbers indicate missing data
    'V161270': 'education', # 1-9 high school; 10-12 some college; 13 bachelors; 14-16 advanced degree
    'V161244': 'church_goer', # 1:'yes',2:'no'
    'V162125x':'patriotism', # how does seeing the flag make you feel? 1:"extremely good", 2:"moderately good", 3:"a little good", 4:"neither good nor bad", 5:"a little bad", 6:"moderately bad", 7:"extremely bad"
    'V161112': 'health_insurance', #  Does R have health insurance? 1:yes, 2:no

    # Political activity and party affiliation
    'V162174': 'discuss_politics', # 1:'yes',2:'no'
    'V162256': 'political_interest', # how interested in politics? 1:"very interested", 2:"somewhat interested", 3:"not very interested", 4:"not at all interested"
    'V161126': 'ideology', # 1:"extremely liberal", 2:"liberal", 3:"slightly liberal", 4:"moderate", 5:"slightly conservative", 6:"conservative", 7:"extremely conservative"
    'V161155': 'pid3', # 1:'Democrat',2:'Republican',3:'Independent'
    'V161158x': 'pid7', # 1:"strong democrat", 2:"not very strong democrat", 3:"independent, but closer to the Democratic party", 4:"independent", 5:"independent, but closer to the Republican party", 6:"not very strong Republican", 7:"strong Republican"
    'V162031x': 'voted', # did you vote in 2016? 0:'no',1:'yes'
    'V162062x': 'votechoice', # 1:"Hillary Clinton", 2:"Donald Trump", 3:"Gary Johnson", 4:"Jill Stein", 5:"Other"

    # Views on select issues
    'V162255': 'obama_muslim', #  is obama a muslim? 1:"yes",2:"no"
    'V161221': 'climate_change', # is global warming happening or not? 1:yes, 2:no
    'V161222': 'cc_caused_by_man', # assuming it's happening, is it caused by humans? 1:mostly human, 2: mostly natural, 3:equally by people and nature
    'V161227': 'same_sex_wedding', # Do you think business owners who provide wedding-related services should be allowed to refuse services to same-sex couples if same-sex marriage violates their religious beliefs, or do you think business owners should be required to provide services regardless of a couple’s sexual orientation? 1:should be allowed to refuse, 2:should be required to provide services
    'V161139': 'econ_health', #  Current economy good or bad? 1: very good ... 5: very bad
    'V161193': 'birthright_citizenship', #  Favor or oppose ending birthright citizenship? 1: favor, 2:oppose, 3:neither
    'V161196': 'build_wall', #  Build a wall with Mexico? 1: favor, 2:oppose, 3:neither
    'V161204': 'affirmative_action', #  Does R favor or oppose affirmative action in universities? 1: favor, 2:oppose, 3:neither
    'V161218': 'corrupt_government', #  How many in government are corrupt? 1:all, 2:most, 3:about half, 4:a few, 5:none
    'V161228': 'bathroom_policy', #  Transgender bathroom policy? 1: have to use bathroom of gender born with; 2: be allowed to use the bathroom of their identified gender
    'V161231': 'gay_marriage', # position on gay marriage? 1: gay couples should be allowed to marry; 2: gay couples should be allowed to form civil unions, but not marry; 3:  There should be no legal recognition of a gay or lesbian couple’s relationship.
    'V161233': 'death_penalty', # R favor oppose death penalty for those convicted of murder? 1: favor, 2: oppose

    # Religious views
    'V161241': 'rel_important', # Is religion important part of R life? 1: important, 2:not important
    'V161242': 'rel_daily_guidance', #  Religion provides guidance in day-to-day living? 1:some, 2:quite a bit, 3:a great deal
    'V161243': 'rel_bible_word', # Is Bible word of God or men?  1: The Bible is the actual word of God and is to be taken literally, word for word. 2:The Bible is the word of God but not everything in it should be taken literally; 3:The Bible is a book written by men and is not the word of God.
    'V161245a': 'rel_attend_church', # Attend church more often than once a week

    # Happy?
    'V161522': 'happy_life' #  How satisfied is R with life? 1:extremely, 2:very; 3:moderately, 4:slightly, 5:not at all

}

In [9]:
issues_columns = [
    "obama_muslim",
    "climate_change",
    "cc_caused_by_man",
    "same_sex_wedding",
    "econ_health",
    "birthright_citizenship",
    "build_wall",
    "affirmative_action",
    "corrupt_government",
    "bathroom_policy",
    "gay_marriage",
    "death_penalty"
]

np.random.seed(42)

In [10]:
# Read in the data ...
import sqlite3
import pandas as pd
conn = sqlite3.connect("anes2016.db")
cursor = conn.cursor()

# print(type(cursor))
# tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

df = pd.read_sql_query("SELECT * FROM anes2016", conn)
# voted = pd.read_sql_query("SELECT voted, age FROM anes2016", conn)
# voted.head()
#conn.close()







cursor.execute('SELECT * FROM anes2016')
test = cursor.fetchall()
test[:5]
df = df.dropna()


# Use the votechoice as y
X = df["happy_life"]
y = df["votechoice"]


# Use only the issues_columns as X
X = df[issues_columns]

In [11]:
# after preparing the data, use train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### Train the Random Forest Classifier Here and extract feature importances...

In [28]:
np.random.seed(42)

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
rf = RandomForestClassifier()
param_dist = {'n_estimators': range(50,500,10), 'max_depth': range(1,20)}
rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, n_iter=5)

rand_search.fit(X_train, y_train)
best_rf = rand_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy is: {accuracy}")


# importances = best_rf.feature_importances_
# print(importances)
# feature_importance = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': importances
# })
# print(feature_importance.sort_values(by='Importance', ascending=False))




c:\Users\caleb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Random Forest Accuracy is: 0.6651053864168618


### Train models and evaluate in the below cell

In [21]:
np.random.seed(42)


# Train the model using the Logistic Regression Algorithm
from sklearn.linear_model import LogisticRegression
param_dist = {'C': [0.1, 1, 10],'solver': ['liblinear', 'saga'], 'max_iter': [100, 200]}
logreg = LogisticRegression()
rand_search = RandomizedSearchCV(logreg, param_distributions=param_dist, n_iter=3, n_jobs=-1)
rand_search.fit(X_train, y_train)
best_logreg = rand_search.best_estimator_
y_pred = best_logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy is: {accuracy}")




c:\Users\caleb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Logistic Regression Accuracy is: 0.6241217798594848


In [ ]:
np.random.seed(42)
# Train the model using the KNN Algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

param_dist = {"n_neighbors": range(3,30)}
knn = KNeighborsClassifier()
rand_search = RandomizedSearchCV(knn, param_distributions= param_dist, n_iter = 5)
rand_search.fit(X_train, y_train)

best_knn = rand_search.best_estimator_
y_pred = best_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN accuracy is: {accuracy}")




c:\Users\caleb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KNN ccuracy is: 0.6416861826697893


In [27]:
np.random.seed(42)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
mlp = MLPClassifier(
    # hidden_layer_sizes=(50, 30),   #started with relu
    # activation='relu',            
    # solver='adam',               
    # max_iter=200,                 
    # random_state=42
)
# y_train = y_train.astype(int)#.squeeze() if data is weird?
# y_test = y_test.astype(int)

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"MLP Accuracy is: {accuracy}")

MLP Accuracy is: 0.6510538641686182


c:\Users\caleb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
np.random.seed(42)